Baseado em: https://github.com/microsoft/recommenders/blob/main/examples/00_quick_start/nrms_MIND.ipynb

<i>Copyright (c) Microsoft Corporation. All rights reserved.</i>

<i>Licensed under the MIT License.</i>

# NRMS: Neural News Recommendation with Multi-Head Self-Attention
NRMS \[1\] is a neural news recommendation approach with multi-head selfattention. The core of NRMS is a news encoder and a user encoder. In the newsencoder, a multi-head self-attentions is used to learn news representations from news titles by modeling the interactions between words. In the user encoder, we learn representations of users from their browsed news and use multihead self-attention to capture the relatedness between the news. Besides, we apply additive
attention to learn more informative news and user representations by selecting important words and news.

## Properties of NRMS:
- NRMS is a content-based neural news recommendation approach.
- It uses multi-self attention to learn news representations by modeling the iteractions between words and learn user representations by capturing the relationship between user browsed news.
- NRMS uses additive attentions to learn informative news and user representations by selecting important words and news.

## Data format:
For quicker training and evaluaiton, we sample MINDdemo dataset of 5k users from [MIND small dataset](https://msnews.github.io/). The MINDdemo dataset has the same file format as MINDsmall and MINDlarge. If you want to try experiments on MINDsmall and MINDlarge, please change the dowload source. Select the MIND_type parameter from ['large', 'small', 'demo'] to choose dataset.
 
**MINDdemo_train** is used for training, and **MINDdemo_dev** is used for evaluation. Training data and evaluation data are composed of a news file and a behaviors file. You can find more detailed data description in [MIND repo](https://github.com/msnews/msnews.github.io/blob/master/assets/doc/introduction.md)

### news data
This file contains news information including newsid, category, subcatgory, news title, news abstarct, news url and entities in news title, entities in news abstarct.
One simple example: <br>

`N46466	lifestyle	lifestyleroyals	The Brands Queen Elizabeth, Prince Charles, and Prince Philip Swear By	Shop the notebooks, jackets, and more that the royals can't live without.	https://www.msn.com/en-us/lifestyle/lifestyleroyals/the-brands-queen-elizabeth,-prince-charles,-and-prince-philip-swear-by/ss-AAGH0ET?ocid=chopendata	[{"Label": "Prince Philip, Duke of Edinburgh", "Type": "P", "WikidataId": "Q80976", "Confidence": 1.0, "OccurrenceOffsets": [48], "SurfaceForms": ["Prince Philip"]}, {"Label": "Charles, Prince of Wales", "Type": "P", "WikidataId": "Q43274", "Confidence": 1.0, "OccurrenceOffsets": [28], "SurfaceForms": ["Prince Charles"]}, {"Label": "Elizabeth II", "Type": "P", "WikidataId": "Q9682", "Confidence": 0.97, "OccurrenceOffsets": [11], "SurfaceForms": ["Queen Elizabeth"]}]	[]`
<br>

In general, each line in data file represents information of one piece of news: <br>

`[News ID] [Category] [Subcategory] [News Title] [News Abstrct] [News Url] [Entities in News Title] [Entities in News Abstract] ...`

<br>

We generate a word_dict file to transform words in news title to word indexes, and a embedding matrix is initted from pretrained glove embeddings.

### behaviors data
One simple example: <br>
`1	U82271	11/11/2019 3:28:58 PM	N3130 N11621 N12917 N4574 N12140 N9748	N13390-0 N7180-0 N20785-0 N6937-0 N15776-0 N25810-0 N20820-0 N6885-0 N27294-0 N18835-0 N16945-0 N7410-0 N23967-0 N22679-0 N20532-0 N26651-0 N22078-0 N4098-0 N16473-0 N13841-0 N15660-0 N25787-0 N2315-0 N1615-0 N9087-0 N23880-0 N3600-0 N24479-0 N22882-0 N26308-0 N13594-0 N2220-0 N28356-0 N17083-0 N21415-0 N18671-0 N9440-0 N17759-0 N10861-0 N21830-0 N8064-0 N5675-0 N15037-0 N26154-0 N15368-1 N481-0 N3256-0 N20663-0 N23940-0 N7654-0 N10729-0 N7090-0 N23596-0 N15901-0 N16348-0 N13645-0 N8124-0 N20094-0 N27774-0 N23011-0 N14832-0 N15971-0 N27729-0 N2167-0 N11186-0 N18390-0 N21328-0 N10992-0 N20122-0 N1958-0 N2004-0 N26156-0 N17632-0 N26146-0 N17322-0 N18403-0 N17397-0 N18215-0 N14475-0 N9781-0 N17958-0 N3370-0 N1127-0 N15525-0 N12657-0 N10537-0 N18224-0`
<br>

In general, each line in data file represents one instance of an impression. The format is like: <br>

`[Impression ID] [User ID] [Impression Time] [User Click History] [Impression News]`

<br>

User Click History is the user historical clicked news before Impression Time. Impression News is the displayed news in an impression, which format is:<br>

`[News ID 1]-[label1] ... [News ID n]-[labeln]`

<br>
Label represents whether the news is clicked by the user. All information of news in User Click History and Impression News can be found in news data file.

## Global settings and imports

In [ ]:
!pip install recommenders
!pip install scrapbook
 
import sys
import os
import numpy as np
import zipfile
from tqdm import tqdm
import scrapbook as sb
from tempfile import TemporaryDirectory
import tensorflow as tf
tf.get_logger().setLevel('ERROR') # only show error messages

from recommenders.models.deeprec.deeprec_utils import download_deeprec_resources 
from recommenders.models.newsrec.newsrec_utils import prepare_hparams
from recommenders.models.newsrec.models.nrms import NRMSModel
from recommenders.models.newsrec.io.mind_iterator import MINDIterator
from recommenders.models.newsrec.newsrec_utils import get_mind_data_set

print("System version: {}".format(sys.version))
print("Tensorflow version: {}".format(tf.__version__))


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 339 kB 5.1 MB/s 
     |████████████████████████████████| 310 kB 95.0 MB/s 
     |████████████████████████████████| 61 kB 10.2 MB/s 
     |████████████████████████████████| 14.4 MB 21.3 MB/s 
     |████████████████████████████████| 5.8 MB 80.5 MB/s 
     |████████████████████████████████| 355 kB 98.3 MB/s 
     |████████████████████████████████| 122 kB 96.2 MB/s 
     |████████████████████████████████| 771 kB 98.0 MB/s 
     |████████████████████████████████| 662 kB 86.2 MB/s 
     |████████████████████████████████| 398 kB 94.0 MB/s 
     |████████████████████████████████| 7.6 MB 85.4 MB/s 
     |████████████████████████████████| 182 kB 97.6 MB/s 
  Created wheel for lightfm: filename=lightfm-1.16-cp38-cp38-linux_x86_64.whl size=747400 sha256=cd68a3866156714eabbc65add80ad03de76db9a8599d35bee0b78777c7a01ea0
  Stored in directory: /root/.cache/pip/whee

## Prepare parameters

In [ ]:
epochs = 5
seed = 42
batch_size = 32

# Options: demo, small, large
MIND_type = 'small'

## Download and load data

In [ ]:
tmpdir = TemporaryDirectory()
data_path = tmpdir.name

train_news_file = os.path.join(data_path, 'train', r'news.tsv')
train_behaviors_file = os.path.join(data_path, 'train', r'behaviors.tsv')
valid_news_file = os.path.join(data_path, 'valid', r'news.tsv')
valid_behaviors_file = os.path.join(data_path, 'valid', r'behaviors.tsv')
wordEmb_file = os.path.join(data_path, "utils", "embedding.npy")
userDict_file = os.path.join(data_path, "utils", "uid2index.pkl")
wordDict_file = os.path.join(data_path, "utils", "word_dict.pkl")
yaml_file = os.path.join(data_path, "utils", r'nrms.yaml')

mind_url, mind_train_dataset, mind_dev_dataset, mind_utils = get_mind_data_set(MIND_type)

if not os.path.exists(train_news_file):
    download_deeprec_resources(mind_url, os.path.join(data_path, 'train'), mind_train_dataset)
    
if not os.path.exists(valid_news_file):
    download_deeprec_resources(mind_url, \
                               os.path.join(data_path, 'valid'), mind_dev_dataset)
if not os.path.exists(yaml_file):
    download_deeprec_resources(r'https://recodatasets.z20.web.core.windows.net/newsrec/', \
                               os.path.join(data_path, 'utils'), mind_utils)

100%|██████████| 51.7k/51.7k [00:03<00:00, 14.3kKB/s]
100%|██████████| 30.2k/30.2k [00:02<00:00, 11.3kKB/s]
100%|██████████| 152k/152k [00:02<00:00, 73.6kKB/s]


## Create hyper-parameters

In [ ]:
hparams = prepare_hparams(yaml_file, 
                          wordEmb_file=wordEmb_file,
                          wordDict_file=wordDict_file, 
                          userDict_file=userDict_file,
                          batch_size=batch_size,
                          epochs=epochs,
                          show_step=10)
print(hparams)

HParams object with values {'support_quick_scoring': True, 'dropout': 0.2, 'attention_hidden_dim': 200, 'head_num': 20, 'head_dim': 20, 'filter_num': 200, 'window_size': 3, 'vert_emb_dim': 100, 'subvert_emb_dim': 100, 'gru_unit': 400, 'type': 'ini', 'user_emb_dim': 50, 'learning_rate': 0.0001, 'optimizer': 'adam', 'epochs': 5, 'batch_size': 32, 'show_step': 10, 'title_size': 30, 'his_size': 50, 'data_format': 'news', 'npratio': 4, 'metrics': ['group_auc', 'mean_mrr', 'ndcg@5;10'], 'word_emb_dim': 300, 'model_type': 'nrms', 'loss': 'cross_entropy_loss', 'wordEmb_file': '/tmp/tmplzsqqmqc/utils/embedding.npy', 'wordDict_file': '/tmp/tmplzsqqmqc/utils/word_dict.pkl', 'userDict_file': '/tmp/tmplzsqqmqc/utils/uid2index.pkl'}


## Train the NRMS model

In [ ]:
iterator = MINDIterator

In [ ]:
model = NRMSModel(hparams, iterator, seed=seed)

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
counterfactual_file = 'counterfactual_behaviors.tsv'

In [ ]:
import pandas as pd

counterfactual_train_data = pd.read_table(counterfactual_file)
counterfactual_train_data

,156966,U32426,11/14/2019 11:00:18 AM,N1150 N55846 N13008 N59546 N18030 N46392 N58090 N55743 N57318 N13374 N306 N30160 N54655 N33096 N28088 N32004 N60702,N9506-0 N27838-0
0,156967,U21312,11/9/2019 3:08:20 AM,N38747 N62451 N39376 N46597,N39720-0 N36448-0 N9620-0 N53748-1 N62797-0
1,156968,U21312,11/11/2019 6:38:48 AM,N38747 N62451 N39376 N46597,N39720-0 N36448-0 N9620-0 N53748-1 N62797-0
2,156969,U93847,11/11/2019 5:52:59 AM,N61826 N39993 N58226 N55920 N1267 N35129 N2753...,N35427-0 N38515-0 N57813-1 N44352-0
3,156970,U93847,11/14/2019 6:13:29 AM,N61826 N39993 N58226 N55920 N1267 N35129 N2753...,N35427-0 N38515-0 N57813-1 N44352-0
4,156971,U93847,11/11/2019 5:54:22 AM,N61826 N39993 N58226 N55920 N1267 N35129 N2753...,N35427-0 N38515-0 N57813-1 N44352-0
...,...,...,...,...,...
32357,189324,U14379,11/12/2019 10:21:40 AM,N1569 N11071 N15254 N27448 N52677 N65066 N5518...,N18251-1 N33600-0 N16026-1 N35125-1 N48295-0 N...
32358,189325,U14379,11/12/2019 10:59:20 AM,N1569 N11071 N15254 N27448 N52677 N65066 N5518...,N18251-1 N33600-0 N16026-1 N35125-1 N48295-0 N...
32359,189326,U14379,11/11/2019 5:22:34 AM,N1569 N11071 N15254 N27448 N52677 N65066 N5518...,N18251-1 N33600-0 N16026-1 N35125-1 N48295-0 N...
32360,189327,U14379,11/12/2019 6:26:31 AM,N1569 N11071 N15254 N27448 N52677 N65066 N5518...,N18251-1 N33600-0 N16026-1 N35125-1 N48295-0 N...


In [ ]:
factual_train_data = pd.read_table(train_behaviors_file)
factual_train_data

,1,U13740,11/11/2019 9:05:58 AM,N55189 N42782 N34694 N45794 N18445 N63302 N10414 N19347 N31801,N55689-1 N35729-0
0,2,U91836,11/12/2019 6:11:30 PM,N31739 N6072 N63045 N23979 N35656 N43353 N8129...,N20678-0 N39317-0 N58114-0 N20495-0 N42977-0 N...
1,3,U73700,11/14/2019 7:01:48 AM,N10732 N25792 N7563 N21087 N41087 N5445 N60384...,N50014-0 N23877-0 N35389-0 N49712-0 N16844-0 N...
2,4,U34670,11/11/2019 5:28:05 AM,N45729 N2203 N871 N53880 N41375 N43142 N33013 ...,N35729-0 N33632-0 N49685-1 N27581-0
3,5,U8125,11/12/2019 4:11:21 PM,N10078 N56514 N14904 N33740,N39985-0 N36050-0 N16096-0 N8400-1 N22407-0 N6...
4,6,U19739,11/11/2019 6:52:13 PM,N39074 N14343 N32607 N32320 N22007 N442 N19001...,N21119-1 N53696-0 N33619-1 N25722-0 N2869-0
...,...,...,...,...,...
156959,156961,U21593,11/14/2019 10:24:05 PM,N7432 N58559 N1954 N43353 N14343 N13008 N28833...,N2235-0 N22975-0 N64037-0 N47652-0 N11378-0 N4...
156960,156962,U10123,11/13/2019 6:57:04 AM,N9803 N104 N24462 N57318 N55743 N40526 N31726 ...,N3841-0 N61571-0 N58813-0 N28213-0 N4428-0 N25...
156961,156963,U75630,11/14/2019 10:58:13 AM,N29898 N59704 N4408 N9803 N53644 N26103 N812 N...,N55913-0 N62318-0 N53515-0 N10960-0 N9135-0 N5...
156962,156964,U44625,11/13/2019 2:57:02 PM,N4118 N47297 N3164 N43295 N6056 N38747 N42973 ...,N6219-0 N3663-0 N31147-0 N58363-0 N4107-0 N457...


In [ ]:
with open(train_behaviors_file) as f:
    factual_lines = f.readlines()

with open(counterfactual_file) as f:
    counterfactual_lines = f.readlines()

factual_with_counterfactual_file = 'factual_with_counterfactual.tsv'
with open(factual_with_counterfactual_file, 'w') as f:
    factual_with_counterfactual_lines = factual_lines + counterfactual_lines
    for line in factual_with_counterfactual_lines:
        f.write(line)

In [ ]:
pd.read_table(factual_with_counterfactual_file)

,1,U13740,11/11/2019 9:05:58 AM,N55189 N42782 N34694 N45794 N18445 N63302 N10414 N19347 N31801,N55689-1 N35729-0
0,2,U91836,11/12/2019 6:11:30 PM,N31739 N6072 N63045 N23979 N35656 N43353 N8129...,N20678-0 N39317-0 N58114-0 N20495-0 N42977-0 N...
1,3,U73700,11/14/2019 7:01:48 AM,N10732 N25792 N7563 N21087 N41087 N5445 N60384...,N50014-0 N23877-0 N35389-0 N49712-0 N16844-0 N...
2,4,U34670,11/11/2019 5:28:05 AM,N45729 N2203 N871 N53880 N41375 N43142 N33013 ...,N35729-0 N33632-0 N49685-1 N27581-0
3,5,U8125,11/12/2019 4:11:21 PM,N10078 N56514 N14904 N33740,N39985-0 N36050-0 N16096-0 N8400-1 N22407-0 N6...
4,6,U19739,11/11/2019 6:52:13 PM,N39074 N14343 N32607 N32320 N22007 N442 N19001...,N21119-1 N53696-0 N33619-1 N25722-0 N2869-0
...,...,...,...,...,...
189322,189324,U14379,11/12/2019 10:21:40 AM,N1569 N11071 N15254 N27448 N52677 N65066 N5518...,N18251-1 N33600-0 N16026-1 N35125-1 N48295-0 N...
189323,189325,U14379,11/12/2019 10:59:20 AM,N1569 N11071 N15254 N27448 N52677 N65066 N5518...,N18251-1 N33600-0 N16026-1 N35125-1 N48295-0 N...
189324,189326,U14379,11/11/2019 5:22:34 AM,N1569 N11071 N15254 N27448 N52677 N65066 N5518...,N18251-1 N33600-0 N16026-1 N35125-1 N48295-0 N...
189325,189327,U14379,11/12/2019 6:26:31 AM,N1569 N11071 N15254 N27448 N52677 N65066 N5518...,N18251-1 N33600-0 N16026-1 N35125-1 N48295-0 N...


In [ ]:
%%time
model = NRMSModel(hparams, iterator, seed=seed)
model.fit(train_news_file, counterfactual_file, valid_news_file, valid_behaviors_file)

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)
step 2790 , total_loss: 0.7206, data_loss: 0.8887: : 2794it [05:13,  8.90it/s]
0it [00:00, ?it/s]/usr/local/lib/python3.8/dist-packages/keras/engine/training_v1.py:2067: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
1326it [00:02, 533.10it/s]
2286it [00:18, 120.77it/s]
73152it [00:09, 7669.14it/s]


at epoch 1
train info: logloss loss:0.7205041050777617
eval info: group_auc:0.4906, mean_mrr:0.2094, ndcg@10:0.2748, ndcg@5:0.2077
at epoch 1 , train time: 313.8 eval time: 94.7


step 2790 , total_loss: 0.5918, data_loss: 0.7462: : 2794it [05:07,  9.10it/s]
1326it [00:01, 851.40it/s]
2286it [00:14, 152.47it/s]
73152it [00:09, 7556.62it/s]


at epoch 2
train info: logloss loss:0.5917865971233975
eval info: group_auc:0.4849, mean_mrr:0.2091, ndcg@10:0.2749, ndcg@5:0.2104
at epoch 2 , train time: 307.1 eval time: 90.3


step 2790 , total_loss: 0.4727, data_loss: 0.2915: : 2794it [05:07,  9.09it/s]
1326it [00:01, 898.25it/s]
2286it [00:14, 153.05it/s]
73152it [00:09, 7707.42it/s]


at epoch 3
train info: logloss loss:0.4725730330725582
eval info: group_auc:0.4909, mean_mrr:0.2131, ndcg@10:0.2788, ndcg@5:0.2155
at epoch 3 , train time: 307.2 eval time: 89.6


step 2790 , total_loss: 0.3410, data_loss: 0.0676: : 2794it [05:07,  9.09it/s]
1326it [00:01, 879.41it/s]
2286it [00:15, 152.30it/s]
73152it [00:09, 7600.90it/s]


at epoch 4
train info: logloss loss:0.34104173585041364
eval info: group_auc:0.4907, mean_mrr:0.211, ndcg@10:0.277, ndcg@5:0.2136
at epoch 4 , train time: 307.4 eval time: 90.0


step 2790 , total_loss: 0.2461, data_loss: 0.1773: : 2794it [05:07,  9.10it/s]
1326it [00:01, 899.89it/s]
2286it [00:14, 153.01it/s]
73152it [00:09, 7691.15it/s]


at epoch 5
train info: logloss loss:0.24601239433282762
eval info: group_auc:0.4994, mean_mrr:0.2154, ndcg@10:0.2831, ndcg@5:0.2192
at epoch 5 , train time: 307.2 eval time: 89.8
CPU times: user 17min 10s, sys: 9min 47s, total: 26min 57s
Wall time: 33min 18s


In [ ]:
%%time
model = NRMSModel(hparams, iterator, seed=seed)
model.fit(train_news_file, factual_with_counterfactual_file, valid_news_file, valid_behaviors_file)

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)
step 10180 , total_loss: 1.2096, data_loss: 1.2624: : 10180it [18:51,  9.00it/s]
0it [00:00, ?it/s]/usr/local/lib/python3.8/dist-packages/keras/engine/training_v1.py:2067: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
1326it [00:02, 489.20it/s]
2286it [00:18, 122.62it/s]
73152it [00:09, 7717.43it/s]


at epoch 1
train info: logloss loss:1.209552888566361
eval info: group_auc:0.6257, mean_mrr:0.2832, ndcg@10:0.3739, ndcg@5:0.3089
at epoch 1 , train time: 1131.4 eval time: 95.0


step 10180 , total_loss: 1.1079, data_loss: 1.2139: : 10180it [18:39,  9.09it/s]
1326it [00:01, 869.76it/s]
2286it [00:15, 151.60it/s]
73152it [00:09, 7708.87it/s]


at epoch 2
train info: logloss loss:1.1078636948860934
eval info: group_auc:0.6284, mean_mrr:0.2901, ndcg@10:0.381, ndcg@5:0.3171
at epoch 2 , train time: 1120.0 eval time: 90.5


step 10180 , total_loss: 1.0567, data_loss: 1.6724: : 10180it [18:39,  9.09it/s]
1326it [00:01, 884.62it/s]
2286it [00:15, 151.81it/s]
73152it [00:09, 7647.39it/s]


at epoch 3
train info: logloss loss:1.0567065000358404
eval info: group_auc:0.635, mean_mrr:0.2976, ndcg@10:0.3891, ndcg@5:0.327
at epoch 3 , train time: 1119.8 eval time: 90.4


step 10180 , total_loss: 1.0157, data_loss: 0.7851: : 10180it [18:39,  9.09it/s]
1326it [00:01, 880.22it/s]
2286it [00:15, 151.81it/s]
73152it [00:09, 7769.94it/s]


at epoch 4
train info: logloss loss:1.015690651402785
eval info: group_auc:0.6376, mean_mrr:0.2987, ndcg@10:0.3896, ndcg@5:0.3259
at epoch 4 , train time: 1119.7 eval time: 90.2


step 10180 , total_loss: 0.9794, data_loss: 1.5101: : 10180it [18:40,  9.09it/s]
1326it [00:01, 871.62it/s]
2286it [00:14, 152.48it/s]
73152it [00:09, 7692.04it/s]


at epoch 5
train info: logloss loss:0.9793790151202257
eval info: group_auc:0.6387, mean_mrr:0.2994, ndcg@10:0.3902, ndcg@5:0.3269
at epoch 5 , train time: 1120.3 eval time: 90.3
CPU times: user 40min 47s, sys: 14min 20s, total: 55min 8s
Wall time: 1h 41min 9s


In [ ]:
hparams

HParams object with values {'support_quick_scoring': True, 'dropout': 0.2, 'attention_hidden_dim': 200, 'head_num': 20, 'head_dim': 20, 'filter_num': 200, 'window_size': 3, 'vert_emb_dim': 100, 'subvert_emb_dim': 100, 'gru_unit': 400, 'type': 'ini', 'user_emb_dim': 50, 'learning_rate': 0.0001, 'optimizer': 'adam', 'epochs': 5, 'batch_size': 32, 'show_step': 10, 'title_size': 30, 'his_size': 50, 'data_format': 'news', 'npratio': 4, 'metrics': ['group_auc', 'mean_mrr', 'ndcg@5;10'], 'word_emb_dim': 300, 'model_type': 'nrms', 'loss': 'cross_entropy_loss', 'wordEmb_file': '/tmp/tmplzsqqmqc/utils/embedding.npy', 'wordDict_file': '/tmp/tmplzsqqmqc/utils/word_dict.pkl', 'userDict_file': '/tmp/tmplzsqqmqc/utils/uid2index.pkl'}

In [ ]:
%%time
res_syn = model.run_eval(valid_news_file, valid_behaviors_file)
print(res_syn)


In [ ]:
sb.glue("res_syn", res_syn)

## Save the model

In [ ]:
model_path = os.path.join(data_path, "model")
os.makedirs(model_path, exist_ok=True)

model.model.save_weights(os.path.join(model_path, "nrms_ckpt"))

## Output Predcition File
This code segment is used to generate the prediction.zip file, which is in the same format in [MIND Competition Submission Tutorial](https://competitions.codalab.org/competitions/24122#learn_the_details-submission-guidelines).

Please change the `MIND_type` parameter to `large` if you want to submit your prediction to [MIND Competition](https://msnews.github.io/competition.html).

In [ ]:
group_impr_indexes, group_labels, group_preds = model.run_fast_eval(valid_news_file, valid_behaviors_file)

586it [00:01, 399.64it/s]
236it [00:03, 67.94it/s]
7538it [00:00, 8052.34it/s]


In [ ]:
with open(os.path.join(data_path, 'prediction.txt'), 'w') as f:
    for impr_index, preds in tqdm(zip(group_impr_indexes, group_preds)):
        impr_index += 1
        pred_rank = (np.argsort(np.argsort(preds)[::-1]) + 1).tolist()
        pred_rank = '[' + ','.join([str(i) for i in pred_rank]) + ']'
        f.write(' '.join([str(impr_index), pred_rank])+ '\n')

7538it [00:00, 35200.73it/s]


In [ ]:
f = zipfile.ZipFile(os.path.join(data_path, 'prediction.zip'), 'w', zipfile.ZIP_DEFLATED)
f.write(os.path.join(data_path, 'prediction.txt'), arcname='prediction.txt')
f.close()

## Reference
\[1\] Wu et al. "Neural News Recommendation with Multi-Head Self-Attention." in Proceedings of the 2019 Conference on Empirical Methods in Natural Language Processing and the 9th International Joint Conference on Natural Language Processing (EMNLP-IJCNLP)<br>
\[2\] Wu, Fangzhao, et al. "MIND: A Large-scale Dataset for News Recommendation" Proceedings of the 58th Annual Meeting of the Association for Computational Linguistics. https://msnews.github.io/competition.html <br>
\[3\] GloVe: Global Vectors for Word Representation. https://nlp.stanford.edu/projects/glove/